# Week 7 Lab: Definite Integrals, Sequences, and the Malthusian Trap

## Measuring Total Accumulation

**SCIE1500 - Analytical Methods for Scientists**

---

### Learning Objectives

By the end of this lab, you will be able to:

1. Evaluate definite integrals using the Fundamental Theorem of Calculus
2. Calculate areas between curves (degraded land application)
3. Understand Monte Carlo methods for area estimation
4. Identify arithmetic and geometric sequences
5. Apply sequence formulas to calculate series sums
6. Model the Malthusian Trap: when population outpaces food supply

---

### What to Submit

1. **During Lab:** Complete **Exercise D** and show your results to your lab demonstrator
2. **By Due Date:** Upload screenshots of completed **Exercises D, G, and H**

---

### Required Files

The following data files are loaded automatically from the course repository:
- `SoilLossMap.png` (loaded automatically from GitHub)
- `DegradedLandAreaImage2.png` (loaded automatically from GitHub)
- `PolygonPoints1.csv` (loaded automatically from GitHub)

---

In [ ]:
# === SETUP: Run this cell first ===
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import pandas as pd
import sympy as sp

# Define symbolic variables
x, t, n, i = sp.symbols('x t n i')

print("✓ All packages loaded successfully!")
print("\nThis week: Definite integrals, sequences, and the Malthusian Trap")

---

## Part A: The Fundamental Theorem of Calculus

### Building on Week 6

Last week, we learned to find **antiderivatives** (indefinite integrals):

$$\int f(x)\,dx = F(x) + C$$

This week, we answer a more concrete question: **What is the total accumulated quantity between two points?**

### The Definite Integral

The **Fundamental Theorem of Calculus (FTC)** states:

$$\boxed{\int_a^b f(x)\,dx = F(b) - F(a)}$$

where $F'(x) = f(x)$ (i.e., $F$ is any antiderivative of $f$).

**Key Distinction:**

| Indefinite Integral | Definite Integral |
|---------------------|-------------------|
| $\int f(x)\,dx = F(x) + C$ | $\int_a^b f(x)\,dx = \text{a number}$ |
| A family of functions | A specific numerical value |
| Includes $+ C$ | No constant of integration |

### Example A.1: Basic Polynomial

In [ ]:
x = sp.symbols('x')

# Evaluate ∫₁³ x² dx
integral1 = sp.integrate(x**2, (x, 1, 3))
print(f"∫₁³ x² dx = {integral1}")
print(f"Decimal: {float(integral1):.4f}")

# Manual verification using FTC:
# F(x) = x³/3
# F(3) - F(1) = 27/3 - 1/3 = 26/3 ≈ 8.67

### Example A.2: Exponential Function

In [ ]:
x = sp.symbols('x')

# Evaluate ∫₀² eˣ dx
integral2 = sp.integrate(sp.exp(x), (x, 0, 2))
print(f"∫₀² eˣ dx = {integral2}")
print(f"Decimal: {float(integral2):.4f}")

# F(x) = eˣ
# F(2) - F(0) = e² - e⁰ = e² - 1 ≈ 6.39

### Example A.3: Logarithmic Integration

In [ ]:
x = sp.symbols('x')

# Evaluate ∫₁ᵉ (1/x) dx
integral3 = sp.integrate(1/x, (x, 1, sp.E))
print(f"∫₁ᵉ (1/x) dx = {integral3}")

# F(x) = ln|x|
# F(e) - F(1) = ln(e) - ln(1) = 1 - 0 = 1

### Visualizing Area Under a Curve

In [ ]:
# Visualize ∫₀³ x² dx = 9
x_vals = np.linspace(0, 3, 100)
y_vals = x_vals**2

plt.figure(figsize=(8, 5))
plt.plot(x_vals, y_vals, 'b-', linewidth=2, label=r'$y = x^2$')

# Shade area under curve
plt.fill_between(x_vals, y_vals, alpha=0.3, color='blue', label='Area = 9')

plt.xlabel('x')
plt.ylabel('y')
plt.title(r'Area under $y = x^2$ from $x=0$ to $x=3$')
plt.legend()
plt.grid(True, alpha=0.3)
plt.axhline(y=0, color='k', linewidth=0.5)
plt.tight_layout()
plt.show()

# Verify with SymPy
x = sp.symbols('x')
area = sp.integrate(x**2, (x, 0, 3))
print(f"Exact area: {area}")

---

## Part B: Why Calculate Areas? — Land Degradation Context

Area analysis is critical for environmental management. The map below shows soil loss due to wind erosion in Australia.

**Questions to consider:**
- Where is soil loss highest (dark red regions)?
- Where is soil being deposited (blue regions)?
- How might we estimate the total affected area?

In [ ]:
# Load and display the soil loss map
try:
    image = mpimg.imread("https://raw.githubusercontent.com/ahailu95/scie1500-content/main/SCIE1500Materials/Week_7/LabFiles/SoilLossMap.png")
    plt.figure(figsize=(12, 12))
    plt.imshow(image)
    plt.title("Estimated Net Soil Loss Due to Wind Erosion (22-23 Sep 2009)")
    plt.axis('off')
    plt.show()
except FileNotFoundError:
    print("⚠️ SoilLossMap.png not found. Please download the image file.")

### Loading the Degraded Land Image

In [ ]:
# Load degraded area image
try:
    image = mpimg.imread("https://raw.githubusercontent.com/ahailu95/scie1500-content/main/SCIE1500Materials/Week_7/LabFiles/DegradedLandAreaImage2.png")
    plt.figure(figsize=(10, 10))
    plt.imshow(image)
    plt.title("Degraded Land Area Measurement Problem")
    plt.axis('off')
    plt.show()
except FileNotFoundError:
    print("⚠️ DegradedLandAreaImage2.png not found.")

---

## Part C: Area Calculation Using Integration (Method 1)

### Loading Boundary Data

The CSV file contains coordinates of points along the boundaries of a degraded area.

In [ ]:
# Load boundary coordinates
try:
    polydata = pd.read_csv("https://raw.githubusercontent.com/ahailu95/scie1500-content/main/SCIE1500Materials/Week_7/LabFiles/PolygonPoints1.csv")
    print("Boundary data loaded successfully!")
    print(f"Shape: {polydata.shape}")
    display(polydata.head(10))
except FileNotFoundError:
    print("⚠️ PolygonPoints1.csv not found.")
    # Create sample data for demonstration
    polydata = pd.DataFrame({
        'boundary': ['A_1', 'A_2', 'A_3', 'B_1', 'B_2', 'B_3'],
        'xcoord': [10, 30, 50, 10, 30, 50],
        'ycoord': [5, 15, 40, 20, 45, 70]
    })

In [ ]:
# Plot the boundary coordinates
xcoord = polydata['xcoord']
ycoord = polydata['ycoord']

plt.figure(figsize=(8, 8))
plt.plot(xcoord, ycoord, 'b-', linewidth=1)
plt.title("Degraded Area Boundary Coordinates")
plt.xlabel("Kilometers")
plt.ylabel("Kilometers")
plt.grid(True, alpha=0.3)
plt.show()

### Fitting Curves to Boundaries

Suppose the boundaries can be approximated by:

- **Lower boundary:** $f(x) = e^{0.0471x}$
- **Upper boundary:** $g(x) = -19.96 + 2.999x - 0.02x^2$

The area between these curves from $x = a$ to $x = b$ is:

$$\text{Area} = \int_a^b [g(x) - f(x)]\,dx$$

In [ ]:
# Define boundary functions
x_sym = sp.symbols('x')

# Lower boundary (exponential)
f_lower = sp.exp(0.0471*x_sym)

# Upper boundary (quadratic)
f_upper = -19.96 + 2.999*x_sym - 0.02*x_sym**2

# Calculate area from x = 7.4 to x = 94.5 (approximate boundary range)
a, b = 7.4, 94.5
area = sp.integrate(f_upper - f_lower, (x_sym, a, b))

print(f"Degraded Land Area Calculation")
print("=" * 40)
print(f"Lower boundary: f(x) = e^(0.0471x)")
print(f"Upper boundary: g(x) = -19.96 + 2.999x - 0.02x²")
print(f"Integration limits: x ∈ [{a}, {b}]")
print(f"\nArea = {float(area.evalf()):.1f} km²")

In [ ]:
# Visualize the boundaries and shaded area
x_vals = np.linspace(0, 100, 200)
y_lower = np.exp(0.0471 * x_vals)
y_upper = -19.96 + 2.999*x_vals - 0.02*x_vals**2

plt.figure(figsize=(10, 8))

# Plot boundaries
plt.plot(x_vals, y_lower, 'r-', linewidth=2, label=r'Lower: $f(x) = e^{0.0471x}$')
plt.plot(x_vals, y_upper, 'g-', linewidth=2, label=r'Upper: $g(x) = -19.96 + 2.999x - 0.02x^2$')

# Shade the area
x_fill = np.linspace(7.4, 94.5, 100)
y_lower_fill = np.exp(0.0471 * x_fill)
y_upper_fill = -19.96 + 2.999*x_fill - 0.02*x_fill**2
plt.fill_between(x_fill, y_lower_fill, y_upper_fill, alpha=0.3, color='brown',
                 label=f'Degraded Area ≈ {float(area.evalf()):.0f} km²')

plt.xlabel('x (km)')
plt.ylabel('y (km)')
plt.title('Degraded Land Area - Integration Method')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim(0, 100)
plt.ylim(0, 100)
plt.tight_layout()
plt.show()

---

## Part D: Monte Carlo Method for Area Estimation (Method 2)

**Monte Carlo methods** are useful when boundaries are too irregular to fit with equations.

### The Algorithm:

1. Put the irregular region inside a rectangle of known area
2. Randomly generate many points inside the rectangle
3. Count how many points fall inside the irregular region
4. Estimate: $\text{Area}_1 \approx \frac{\text{points inside}}{\text{total points}} \times \text{Area}_{\text{rectangle}}$

The more points you use, the more accurate the estimate!

In [ ]:
# Simple Monte Carlo demonstration: Estimate area of a circle
# True area of unit circle = π ≈ 3.14159

np.random.seed(42)  # For reproducibility
n_points = 10000

# Generate random points in a 2×2 square centered at origin
x_rand = np.random.uniform(-1, 1, n_points)
y_rand = np.random.uniform(-1, 1, n_points)

# Check which points are inside the unit circle (x² + y² ≤ 1)
inside_circle = (x_rand**2 + y_rand**2) <= 1

# Estimate area
proportion_inside = np.sum(inside_circle) / n_points
area_square = 4  # 2 × 2 = 4
estimated_area = proportion_inside * area_square

print(f"Monte Carlo Estimation of π")
print("=" * 40)
print(f"Points generated: {n_points}")
print(f"Points inside circle: {np.sum(inside_circle)}")
print(f"Proportion inside: {proportion_inside:.4f}")
print(f"Estimated area: {estimated_area:.4f}")
print(f"True area (π): {np.pi:.4f}")
print(f"Error: {abs(estimated_area - np.pi):.4f}")

In [ ]:
# Visualize the Monte Carlo simulation
plt.figure(figsize=(8, 8))

# Plot points (colored by inside/outside)
plt.scatter(x_rand[inside_circle], y_rand[inside_circle], 
            c='blue', s=1, alpha=0.5, label='Inside circle')
plt.scatter(x_rand[~inside_circle], y_rand[~inside_circle], 
            c='red', s=1, alpha=0.5, label='Outside circle')

# Draw the circle
theta = np.linspace(0, 2*np.pi, 100)
plt.plot(np.cos(theta), np.sin(theta), 'k-', linewidth=2)

plt.xlabel('x')
plt.ylabel('y')
plt.title(f'Monte Carlo Estimation: π ≈ {estimated_area:.4f}')
plt.legend(loc='upper right')
plt.axis('equal')
plt.grid(True, alpha=0.3)
plt.show()

### Convergence: More Points → Better Accuracy

In [ ]:
# Show convergence of Monte Carlo estimate
np.random.seed(123)

n_values = [100, 500, 1000, 5000, 10000, 50000, 100000]
estimates = []

for n in n_values:
    x_rand = np.random.uniform(-1, 1, n)
    y_rand = np.random.uniform(-1, 1, n)
    inside = (x_rand**2 + y_rand**2) <= 1
    estimate = 4 * np.sum(inside) / n
    estimates.append(estimate)
    print(f"n = {n:>6}: π ≈ {estimate:.5f} (error = {abs(estimate - np.pi):.5f})")

# Plot convergence
plt.figure(figsize=(10, 5))
plt.semilogx(n_values, estimates, 'bo-', markersize=8, label='MC Estimate')
plt.axhline(y=np.pi, color='r', linestyle='--', label=f'True π = {np.pi:.5f}')
plt.xlabel('Number of Points (log scale)')
plt.ylabel('Estimated Value of π')
plt.title('Monte Carlo Convergence')
plt.legend()
plt.grid(True, alpha=0.3)
plt.show()

---

## Part E: Arithmetic and Geometric Sequences

### Definitions

**Arithmetic Sequence:** Each term differs from the previous by a constant **difference** $d$.
$$a_n = a_1 + (n-1)d$$

**Geometric Sequence:** Each term is multiplied by a constant **ratio** $r$.
$$a_n = a_1 \cdot r^{n-1}$$

| Sequence Type | Example | Pattern |
|---------------|---------|--------|
| Arithmetic | 2, 5, 8, 11, 14, ... | Add 3 each time ($d = 3$) |
| Geometric | 3, 6, 12, 24, 48, ... | Multiply by 2 each time ($r = 2$) |

### Example E.1: Generating Sequences

In [ ]:
def generate_arithmetic_seq(a1, d, n):
    """Generate arithmetic sequence: a_i = a1 + (i-1)*d"""
    return [a1 + d*(i-1) for i in range(1, n+1)]

def generate_geometric_seq(a1, r, n):
    """Generate geometric sequence: a_i = a1 * r^(i-1)"""
    return [a1 * r**(i-1) for i in range(1, n+1)]

# Examples
print("Arithmetic sequence (a1=2, d=3):")
print(generate_arithmetic_seq(2, 3, 10))

print("\nGeometric sequence (a1=3, r=2):")
print(generate_geometric_seq(3, 2, 10))

### Exercise: Identify the Sequence Type

Determine whether each sequence is arithmetic or geometric:

1. 0, 6, 12, ...
2. 1, 2, 4, ...
3. 2, 4, 6, ...
4. -8, 4, -2, ...
5. 100, 102, 104.04, ...

In [ ]:
# Let's check each sequence
sequences = [
    [0, 6, 12],
    [1, 2, 4],
    [2, 4, 6],
    [-8, 4, -2],
    [100, 102, 104.04]
]

for i, seq in enumerate(sequences, 1):
    # Check for arithmetic (constant difference)
    diff1 = seq[1] - seq[0]
    diff2 = seq[2] - seq[1]
    
    # Check for geometric (constant ratio)
    if seq[0] != 0:
        ratio1 = seq[1] / seq[0]
        ratio2 = seq[2] / seq[1]
    else:
        ratio1 = ratio2 = None
    
    print(f"Sequence {i}: {seq}")
    print(f"  Differences: {diff1}, {diff2}")
    if ratio1:
        print(f"  Ratios: {ratio1}, {ratio2}")
    
    if abs(diff1 - diff2) < 0.001:
        print(f"  → ARITHMETIC (d = {diff1})")
    elif ratio1 and abs(ratio1 - ratio2) < 0.001:
        print(f"  → GEOMETRIC (r = {ratio1})")
    print()

### Series Sums

**Arithmetic Series Sum:**
$$S_n = \frac{n}{2}(a_1 + a_n) = \frac{n}{2}[2a_1 + (n-1)d]$$

**Geometric Series Sum:**
$$S_n = a_1 \cdot \frac{1 - r^n}{1 - r} \quad (r \neq 1)$$

In [ ]:
# Sum of first 12 terms of 2, 4, 6, 8, ... (arithmetic, a1=2, d=2)
a1, d, n = 2, 2, 12
seq = generate_arithmetic_seq(a1, d, n)
print(f"Arithmetic sequence: {seq}")
print(f"Direct sum: {sum(seq)}")

# Using formula: S_n = n/2 * (a1 + a_n)
a_n = a1 + (n-1)*d
S_n = n/2 * (a1 + a_n)
print(f"Formula sum: {S_n}")

### Q29-Style: Counting Terms in a Range

**Problem:** Consider the sequence $a_i = 3 + 5(i-1)$ for $i = 1, 2, 3, ...$

How many terms are between 10 and 150 (inclusive)?

In [ ]:
# Sequence: a_i = 3 + 5(i-1) = 5i - 2
# This is arithmetic with a1 = 3, d = 5

a1, d = 3, 5
lower, upper = 10, 150

# Find smallest i where a_i >= 10
# 3 + 5(i-1) >= 10
# 5(i-1) >= 7
# i >= 7/5 + 1 = 2.4
# So i_min = 3

i_min = int(np.ceil((lower - a1) / d + 1))

# Find largest i where a_i <= 150
# 3 + 5(i-1) <= 150
# 5(i-1) <= 147
# i <= 147/5 + 1 = 30.4
# So i_max = 30

i_max = int(np.floor((upper - a1) / d + 1))

count = i_max - i_min + 1

print(f"Sequence: a_i = {a1} + {d}(i-1)")
print(f"Range: [{lower}, {upper}]")
print(f"Smallest index with a_i >= {lower}: i = {i_min}")
print(f"Largest index with a_i <= {upper}: i = {i_max}")
print(f"\n✓ Number of terms in range: {count}")

# Verify
print(f"\nVerification:")
print(f"a_{i_min} = {a1 + (i_min-1)*d} (should be >= {lower})")
print(f"a_{i_max} = {a1 + (i_max-1)*d} (should be <= {upper})")

---

## Part F: The Malthusian Trap

**Thomas Malthus (1798)** predicted a crisis:
- **Population** grows **geometrically** (exponentially)
- **Food production** grows **arithmetically** (linearly)

Eventually, population will exceed food capacity → famine, disease, crisis.

### Model Setup

Let's model a hypothetical country:
- Initial population: $P_0 = 100$ million
- Population growth rate: 3% per year → $P_n = 100 \cdot 1.03^n$
- Initial food capacity: $F_0 = 100$ million people
- Food capacity growth: 5 million per year → $F_n = 100 + 5n$

In [ ]:
# Malthusian Trap Parameters
P0 = 100  # Initial population (millions)
F0 = 100  # Initial food capacity (millions)
r = 1.03  # Population growth rate (3% per year)
d = 5     # Food capacity increase per year (millions)
years = 100

# Generate sequences
n = np.arange(years + 1)
population = P0 * r**n            # Geometric: P_n = P0 * r^n
food_capacity = F0 + d * n         # Arithmetic: F_n = F0 + d*n

# Find crisis point (where population exceeds food capacity)
crisis_idx = np.where(population > food_capacity)[0]
if len(crisis_idx) > 0:
    crisis_year = crisis_idx[0]
    print(f"MALTHUSIAN TRAP ANALYSIS")
    print("=" * 40)
    print(f"Crisis year: {crisis_year}")
    print(f"Population at crisis: {population[crisis_year]:.1f} million")
    print(f"Food capacity at crisis: {food_capacity[crisis_year]:.1f} million")
else:
    print("No crisis within simulation period")
    crisis_year = None

In [ ]:
# Visualize the Malthusian Trap
plt.figure(figsize=(10, 6))
plt.plot(n, population, 'r-', linewidth=2, label='Population (geometric, 3% growth)')
plt.plot(n, food_capacity, 'g-', linewidth=2, label='Food Capacity (arithmetic, +5M/year)')

if crisis_year:
    plt.axvline(x=crisis_year, color='orange', linestyle='--', 
                label=f'Crisis at year {crisis_year}')
    plt.scatter([crisis_year], [population[crisis_year]], 
                color='red', s=100, zorder=5)

plt.xlabel('Years')
plt.ylabel('Population / Food Capacity (millions)')
plt.title('Malthusian Trap: Geometric Population vs Arithmetic Food Growth')
plt.legend()
plt.grid(True, alpha=0.3)
plt.xlim([0, 80])
plt.tight_layout()
plt.show()

In [ ]:
# Create a data frame to see the values
df = pd.DataFrame({
    'Year': n,
    'Population': population,
    'Food_Capacity': food_capacity
})

# Show years around the crisis point
if crisis_year:
    print("\nValues around the crisis point:")
    display(df.iloc[crisis_year-3:crisis_year+3])

---

## 📝 STUDENT EXERCISE D (Show Demonstrator)

### Degraded Land Area Calculation

Given the boundaries:
- **Lower boundary:** $f(x) = e^{0.0471x}$
- **Upper boundary:** $g(x) = -19.96 + 2.999x - 0.02x^2$

**Tasks:**
1. Plot both boundaries on the same graph (x from 0 to 100)
2. Shade the area between the curves from $x = 7.4$ to $x = 94.5$
3. Calculate the exact area using SymPy integration
4. Label your plot with the area value

**Color Requirements:**
- Lower boundary: RED
- Upper boundary: GREEN
- Shaded area: Use a brown or tan color

In [ ]:
# EXERCISE D: Your code here

# Step 1: Calculate the area using SymPy
x = sp.symbols('x')
# f_lower = 
# f_upper = 
# area = sp.integrate(...)
# print(f"Area = {float(area.evalf()):.1f} km²")

In [ ]:
# EXERCISE D: Create the plot
# plt.figure(figsize=(10, 8))
# Plot curves...
# Shade area...
# Add labels...

---

## 📝 STUDENT EXERCISE G (Upload)

### Modified Malthusian Trap

Suppose healthcare improvements increase the population growth rate to **3.5%** per year (instead of 3%).

Food production growth remains unchanged (+5 million/year).

**Tasks:**
1. Generate a new population sequence with 3.5% growth rate
2. Plot both population and food capacity on the same graph
3. Determine the new crisis year
4. Fill in: "If the population growth rate is **3.5%**, the country would face food shortages within **___ years** and its population would be **___ million** by then."

In [ ]:
# EXERCISE G: Modified Malthusian Trap

# New growth rate
r_new = 1.035  # 3.5% growth

# Generate new population sequence
# population2 = P0 * r_new**n

# Find new crisis point
# crisis_idx2 = np.where(population2 > food_capacity)[0]

# Print results
# print(f"New crisis year: ...")
# print(f"Population at crisis: ... million")

In [ ]:
# EXERCISE G: Plot the new scenario
# plt.figure(figsize=(10, 6))
# Plot population2 and food_capacity...
# Mark the crisis point...

**TIMING OF MALTHUSIAN TRAP 2:** If the population growth rate is higher at **3.5%**, the country would face food shortages within **___ years** and its population size would be **___ million** by then.

---

## 📝 STUDENT EXERCISE H (Upload)

### Sequence and Series Practice Quiz

Answer the following questions and show your work:

**Q1.** For the sequence $a_i = 3 + 5(i-1)$, find $a_{10}$.

**Q2.** What is the sum of the first 20 terms of the arithmetic sequence 2, 5, 8, 11, ...?

**Q3.** For the geometric sequence 2, 6, 18, 54, ..., what is the common ratio $r$?

**Q4.** If a population starts at 1000 and grows at 5% per year, what is the population after 10 years?

**Q5.** What is the definition of a geometric sequence?

In [ ]:
# EXERCISE H: Show your calculations

# Q1: a_10 = 3 + 5(10-1) = ?

# Q2: Sum of 2, 5, 8, 11, ... (first 20 terms)
# a1 = 2, d = 3, n = 20
# S_n = n/2 * (2*a1 + (n-1)*d) = ?

# Q3: r = 6/2 = ?

# Q4: P = 1000 * 1.05^10 = ?

# Q5: Definition (write in markdown cell below)

**Your Answers:**

Q1. Answer: _______________

Q2. Answer: _______________

Q3. Answer: _______________

Q4. Answer: _______________

Q5. Answer: _______________

---

## Summary: Key Formulas

| Concept | Formula |
|---------|---------|
| Definite Integral (FTC) | $\int_a^b f(x)\,dx = F(b) - F(a)$ where $F'(x) = f(x)$ |
| Area under curve | $\text{Area} = \int_a^b f(x)\,dx$ when $f(x) \geq 0$ |
| Area between curves | $\text{Area} = \int_a^b [f(x) - g(x)]\,dx$ |
| Arithmetic sequence | $a_n = a_1 + (n-1)d$ |
| Arithmetic series | $S_n = \frac{n}{2}(a_1 + a_n)$ |
| Geometric sequence | $a_n = a_1 \cdot r^{n-1}$ |
| Geometric series | $S_n = a_1 \frac{1-r^n}{1-r}$ |

### Key Python Commands

| Task | Command |
|------|--------|
| Definite integral | `sp.integrate(expr, (x, a, b))` |
| Fill between curves | `plt.fill_between(x, y_lower, y_upper)` |
| Generate arithmetic seq | `[a1 + d*(i-1) for i in range(1, n+1)]` |
| Generate geometric seq | `[a1 * r**(i-1) for i in range(1, n+1)]` |

---

## What's Next?

**Week 8** will cover:
- Predator-Prey Dynamics (Lotka-Volterra model)
- Systems of interacting populations
- Phase diagrams and equilibrium points

---

*The Malthusian Trap reminds us why mathematics matters: understanding exponential vs. linear growth can help us anticipate and address real-world crises!*